In [1]:
# Importing tools for querying research papers and Wikipedia articles

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun # ArxivQueryRun: A tool from langchain_community for querying research papers from Arxiv.
# WikipediaQueryRun: A tool from langchain_community for querying information from Wikipedia.

from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper # ArxivAPIWrapper: An API wrapper for Arxiv, allowing access to research papers from the Arxiv repository.
# WikipediaAPIWrapper: An API wrapper for Wikipedia, providing a way to fetch detailed information from Wikipedia articles.

In [2]:
# Used the Inbuild tool of Wikipedia
api_wraper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250) # We can increase the content characters
wiki = WikipediaQueryRun(api_wrapper=api_wraper_wiki)
wiki.name

'wikipedia'

In [3]:
# Inbuild Arxiv tool created
api_wraper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wraper_arxiv)
arxiv.name

'arxiv'

In [4]:
tools = [wiki,arxiv]

In [5]:
# Custom tool [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E046108F70>, search_kwargs={})

In [7]:
# To convert retriever into tools
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,'Langsmith-search','Search any information about langchain')
retriever_tool.name

'Langsmith-search'

In [8]:
tools = [wiki,arxiv,retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\15_LangChain\\myenv1\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith-search', description='Search any information about langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E07F317B50>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E046108F70>, search_k

In [10]:
import os
from langchain_groq import ChatGroq

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# Fetching API Keys for Groq API Key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [12]:
# Set API keys to environment variables
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [13]:
# Initialize embeddings using HuggingFace model for vector generation
# from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [14]:
# Initialize the Groq model for generating answers
llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="Llama3-8b-8192")

In [15]:
# Create our Prompt template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [20]:
# Agents to chain 
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [17]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [23]:
agent_executor.invoke({'input':'Tell me about Langsmith'})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation neede

{'input': 'Tell me about Langsmith',
 'output': 'It seems like the tool call "call_ewnb" yielded a webpage about LangSmith, a platform for building production-grade LLM applications. LangSmith seems to provide tools for monitoring and evaluating LLM applications, and integrates seamlessly with LangChain\'s open source frameworks langchain and langgraph. It also provides a SDK for logging traces and running evaluations.\n\nBased on this information, I can provide you with more details about LangSmith. LangSmith allows developers to closely monitor and evaluate their LLM applications, so they can ship quickly and with confidence. It provides a range of features, including:\n\n* Logging traces: LangSmith provides multiple ways to log traces to the platform, including using the traceable() function.\n* Running evaluations: LangSmith allows developers to run evaluations to measure the performance of their LLM applications.\n* Integrating with LangChain: LangSmith integrates seamlessly with 

In [24]:
agent_executor.invoke({'input':'what is machine learning'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`
responded: Based on the result provided by the "wikipedia" tool, it seems that machine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data.

Here's a more detailed response:

Machine learning is a subfield of artificial intelligence that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. The goal of machine learning is to develop systems that can automatically improve 

{'input': 'what is machine learning',
 'output': 'Thank you for providing the output of the tool call. Since the output seems to be a tutorial on how to use Langsmith, I will respond directly with some information about Langsmith.'}

In [25]:
agent_executor.invoke({'input':'cricket champions trophy 2025'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Cricket Champions Trophy 2025'}`


Page: 2025 ICC Champions Trophy
Summary: The 2025 ICC Champions Trophy  will be the ninth edition of the ICC Champions Trophy. It will be hosted by Pakistan and United Arab Emirates from 19 February to 9 March 2025 and will be contested by the top ei
Invoking: `wikipedia` with `{'query': '2025 ICC Champions Trophy'}`


Page: 2025 ICC Champions Trophy
Summary: The 2025 ICC Champions Trophy  will be the ninth edition of the ICC Champions Trophy. It will be hosted by Pakistan and United Arab Emirates from 19 February to 9 March 2025 and will be contested by the top eiThe 2025 ICC Champions Trophy will be the ninth edition of the ICC Champions Trophy. It will be hosted by Pakistan and United Arab Emirates from 19 February to 9 March 2025 and will be contested by the top eight teams in the ICC ODI Championship rankings.

> Finished chain.


{'input': 'cricket champions trophy 2025',
 'output': 'The 2025 ICC Champions Trophy will be the ninth edition of the ICC Champions Trophy. It will be hosted by Pakistan and United Arab Emirates from 19 February to 9 March 2025 and will be contested by the top eight teams in the ICC ODI Championship rankings.'}

In [28]:
agent_executor.invoke({'input':'preacisent of America 2025'})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'United States in 2025'}`


Page: 2025 in the United States
Summary: The following is a list of events of the year 2025 in the United States, as well as predicted and scheduled events that have not yet occurred. The start of the year has been dominated by the January 2025 South
Invoking: `wikipedia` with `{'query': 'January 2025 South'}`


Page: 2025
Summary: 2025 (MMXXV) is the current year, and is a common year starting on Wednesday of the Gregorian calendar, the 2025th year of the Common Era (CE) and Anno Domini (AD) designations, the 25th  year of the 3rd millennium and the 21st ce
Invoking: `wikipedia` with `{'query': 'January 2025 South'}`


Page: 2025
Summary: 2025 (MMXXV) is the current year, and is a common year starting on Wednesday of the Gregorian calendar, the 2025th year of the Common Era (CE) and Anno Domini (AD) designations, the 25th  year of the 3rd millennium and the 21st ce
Invoking: `wik

{'input': 'preacisent of America 2025',
 'output': "It seems like the tool call yielded the same result again. Let me try to provide a text response instead.\n\nIt appears that the tool call is stuck in a loop, providing the same result again. If you're looking for information on January 2025, I can try to provide a more general overview. January 2025 is expected to be a significant month, with the potential for significant weather events and economic shifts. However, more specific information is not yet available as we are still in the future. Would you like me to provide more general information on the topic?"}